# How to use the Seshat API

First import the package and set up a client.

In [1]:
from seshat_api import SeshatAPI
import pandas as pd

In [2]:
client = SeshatAPI(base_url="http://localhost:8000/api")  # Use this if you are running the API locally
# client = SeshatAPI(base_url="https://seshatdata.com/api")  # Use this if you are using the public API (seshatdata is a test server)

In [3]:
# Function to get all pages of data from the API for a given function
def get_seshat_data(client, data_func):
    data = data_func(client)
    all_results = []
    page = 1

    while True:
        try:
            response = data.get_page(page)
        except:
            break
        results = response.get('results', [])
        if not results:
            break
        all_results.extend(results)
        page += 1

    return pd.DataFrame(all_results)

As a first test, let's import `Polities` from the `core` app and look at them in pandas:

In [4]:
from seshat_api.core import Polities
polities_df = get_seshat_data(client, Polities)

In [ ]:
polities_df.head()

Now let's try something more interesting! I want to know which polities were Christian in the year 1500 and add them to a pandas dataframe.

In [6]:
from seshat_api.core import Religions
religions_df = get_seshat_data(client, Religions)

In [ ]:
religions_df.head()

In [8]:
# Using polities and religions data and matching on the id field, add the polity.long_name and religion.religion_name to a new df, for all polities that exist in 1500 based on the start_year and end_year
# df = pd.merge(polities_df, religions_df, left_on='id', right_on='id', how='inner')
# df = df[(df['start_year'] <= 1500) & (df['end_year'] >= 1500)]
# df = df[['long_name', 'religion_name']]
# df

In [9]:
# from seshat_api.core import CliopatriaShapefiles
# cliopatria_df = get_seshat_data(client, CliopatriaShapefiles)

In [ ]:
# cliopatria_df.head()